<a href="https://colab.research.google.com/github/SaibalPatraDS/Hands-on-LLM/blob/main/Intro_to_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro to Prompt Engineering

In [ ]:
!pip install torch transformers>=4.40.1 datasets>=2.18.0 langchain>=0.1.17

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### importing necessary libraries

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
## loading the llm and tokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map = 'cuda',
    torch_dtype = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct"
)

## create a pipeline to use the model
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_tokens = 500,
    do_sample = False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
## prompt
messages = [
    {
        "role" : "user",
        "content" : "Write a joke about the black color!"
    }
]

## generate the text
response = pipe(messages)
print(response[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why don't black cats play poker?

Because they're afraid of getting caught with a "paws"!


In [ ]:
## Apply Prompt Template
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize = False)
print(prompt)

<|user|>
Write a joke about the black color!<|end|>
<|endoftext|>


In [ ]:
## Using a high temperature
output = pipe(messages, do_sample = True, temperature = 1)
print(output[0]["generated_text"])

 I told my friend I had a new piece of artwork. She saw it and said, "Wow, that's beautiful!" I replied, "Oh, thank you, it's a painting of the night sky. It's truly the 'black' of the best kind!"


In [ ]:
## Using high p
output = pipe(messages, do_sample = True, top_p = 1)
print(output[0]["generated_text"])

 Why don't some people believe in love?

Because they think you should only see red in colors!


## Advanced Prompt Engineering

In [ ]:
# Text to summarize which we stole from https://jalammar.github.io/illustrated-transformer/ ;)
text = """In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud’s recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let’s try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard’s NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let’s begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other – there’s nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder’s inputs first flow through a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We’ll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we’ve seen the major components of the model, let’s start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 – In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below. The size of this list is hyperparameter we can set – basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we’ll switch up the example to a shorter sentence and we’ll look at what happens in each sub-layer of the encoder.
Now We’re Encoding!
As we’ve mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a ‘self-attention’ layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""

# Prompt components
persona = "You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.\n"
instruction = "Summarize the key findings of the paper provided.\n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.\n"
data_format = "Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.\n"
audience = "The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.\n"
tone = "The tone should be professional and clear.\n"
#text = "MY TEXT TO SUMMARIZE"  # Replace with your own text to summarize
data = f"Text to summarize: {text}"

# The full prompt - remove and add pieces to view its impact on the generated output
query = persona + instruction + data_format + audience + tone + data

In [ ]:
messages = [
    {"role": "user", "content": query}
]
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided.
Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.
The tone should be professional and clear.
Text to summarize: In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelizati

In [ ]:
# Generate the output
outputs = pipe(messages)
print(outputs[0]["generated_text"])

 **Method Summary:**

- The Transformer model uses attention mechanisms to improve the training speed of deep learning models.
- It consists of an encoding component and a decoding component, each with multiple identical sub-layers.
- Encoders process input vectors through self-attention and feed-forward neural networks.
- Decoders use self-attention and focus on relevant parts of the input sentence.
- Embedding transforms input words into vectors, which are then processed by the encoders.
- The model allows for parallel processing in the feed-forward layers, enhancing training efficiency.

**Main Results:**

The Transformer model, as detailed in the paper "Attention is All You Need," represents a significant advancement in the field of deep learning, particularly in the realm of neural machine translation. By leveraging attention mechanisms, the model achieves superior performance compared to the Google Neural Machine Translation model for specific tasks. A key feature of the Transfor

## In Context Prompt Engineering

In [ ]:
## Example of One Shot Learning [We give the model an example to learn from and then expect it to perform accroding to that]

messages = [
    {
        "role" : "user",
        "content" : "We have to find the next word of the sequence : FAG, GAF, HAI, IAH, ____. No Explanation is Required."},

    {
        "role" : "assistant",
        "content" : "Next word will be JAK."
    },

    {
        "role" : "user",
        "content" : "Find the missing word : SCD, TEF, UGH, VIJ, ___. No Explanation is Required."
    }
]

## checking whether all info all clearly stored
print(tokenizer.apply_chat_template(messages, tokenize = False))

<|user|>
We have to find the next word of the sequence : FAG, GAF, HAI, IAH, ____. No Explanation is Required.<|end|>
<|assistant|>
Next word will be JAK.<|end|>
<|user|>
Find the missing word : SCD, TEF, UGH, VIJ, ___. No Explanation is Required.<|end|>
<|endoftext|>


In [ ]:
## Generate the Output
output = pipe(messages)
print(output[0]["generated_text"])

 The missing word is WKL.


## Chain Prompting - Breaking the problem into smaller chunks

Task - Suggest name of the product on the basis of features and suggest slogan of the campaign. And Generate a short sales pitch for the same product.

In [ ]:
## Suggest Name and Slogan for the product

messages = [
    {
        "role" : "user",
        "content" : "Suggest a creative name for a chatbot that can used to learn Laws and Financial Literature and also suggest a slogan for the same. \
        Result should look like \
        Name : \
        Slogan : "
    }
]

## Looking into the results
product = pipe(messages, temperature = 1)
print(product[0]["generated_text"])

 Name: LegalEagleLearn

Slogan: "Empowering your financial and legal knowledge with precision and ease."


In [ ]:
## Suggesting Sales Pitch for the same product
sales_messages = [
    {
        "role" : "user",
        "content" : f"Write a sales pitch in short for the product in not more than 200 words and key points to be in bullet points, '{product}'"
    }
]

## looking into the sales pitch
sales_pitch = pipe(sales_messages)
print(sales_pitch[0]["generated_text"])

 Introducing LegalEagleLearn, the ultimate online platform designed to empower your financial and legal knowledge with precision and ease. Our comprehensive courses cover a wide range of topics, from basic legal principles to advanced financial strategies, ensuring you have the expertise to navigate the complex world of law and finance.

Key Points:
- Accessible and affordable: LegalEagleLearn offers flexible learning options, allowing you to study at your own pace and on your own schedule.
- Expert-led courses: Our courses are taught by experienced professionals who have years of experience in the legal and financial fields.
- Interactive learning: Engage with our interactive learning tools, including quizzes, case studies, and real-life scenarios to enhance your understanding.
- Certification: Upon completion of a course, you will receive a certificate of achievement, which can be added to your resume or LinkedIn profile.
- Community support: Join our vibrant community of learners an

## Reasoning with Generative Models


----

*  LLMS are tend to think before answering anything

------------------------------------------------------

### 1. Chain of Thoughts Prompting [Prompts for Reasoning]